In [2]:
from __future__ import print_function
import numpy as np
import tensorflow as tf

In [29]:

batch_size = 100
kappaNN = 0.001*2
kappaLGSTC = 0.001*5

numwords = 3
vocab_size = 250
n_embed = 150
n_hidden = 100
n_output = n_embed

graph = tf.Graph()
with graph.as_default():
    
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.int32, shape=(batch_size, numwords))
    tf_train_labels = tf.placeholder(tf.int32, shape=(batch_size, 1))
    #tf_valid_dataset = tf.constant(valid_dataset)
    #tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weightsW2E = tf.Variable(tf.truncated_normal([vocab_size, n_embed]))
    weightsE2H = tf.Variable(tf.truncated_normal([n_embed*numwords, n_hidden]))
    biasesH = tf.Variable(tf.zeros([n_hidden]))
    weightsH2O = tf.Variable(tf.truncated_normal([n_hidden, n_output]))
    biasesO = tf.Variable(tf.zeros([n_output]))

    # Training computation.
    train_embed = tf.gather(weightsW2E, tf_train_dataset, validate_indices=None, name=None)
    train_logitsH = tf.matmul(train_embed, weightsE2H) + biasesH
    train_H = tf.sigmoid(train_logitsH)
    train_logitsO = tf.matmul(train_H, weightsH2O) + biasesO
    #train_predictionO = tf.nn.softmax(train_logitsO) # Reference only : not used in cross-entropy function

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(train_logitsO, tf_train_labels))
    param1_l2 = tf.nn.l2_loss(weightsE2H)# + tf.nn.l2_loss(biasesH)
    param2_l2 = tf.nn.l2_loss(weightsH2O)# + tf.nn.l2_loss(biasesO)
    reg_loss = loss + kappaNN * (param1_l2 + param2_l2)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(reg_loss)

    '''
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(train_logitsO)
    #
    valid_logitsH = tf.matmul(tf_valid_dataset, weightsE2H) + biasesE2H
    valid_H = tf.nn.relu(valid_logitsH)
    valid_logitsO = tf.matmul(valid_H, weightsH2O) + biasesH2O
    valid_predictionO = tf.nn.softmax(valid_logitsO)
    #
    test_logitsH = tf.matmul(tf_test_dataset, weightsE2H) + biasesE2H
    test_H = tf.nn.relu(test_logitsH)
    test_logitsO = tf.matmul(test_H, weightsH2O) + biasesH2O
    test_predictionO = tf.nn.softmax(test_logitsO)'''


ValueError: Shape must be rank 2 but is rank 3 for 'MatMul' (op: 'MatMul') with input shapes: [100,3,150], [450,100].

In [23]:
num_steps = 10 # to debug

from utilities import load_data

[train_input, train_target, valid_input, valid_target, test_input, test_target, vocab] = load_data(batch_size)

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        #offset = step % 2
        # Generate a minibatch.
        batch_data = tf.to_int32(train_input[:,:,step])
        batch_labels = tf.to_int32(train_target[:,:,step])
        #batch_data = train_dataset[offset:(offset + batch_size), :]
        #batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized


NameError: name 'optimizer' is not defined

In [9]:
print(train_input.shape)
print(train_target.shape)

(3, 100, 3725)
(1, 100, 3725)
